# The feedforward neural network model for learning PageRank

In [1]:
import time
from dihelpers import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [3]:
# Pageank as the target or label data
target_pagerank = np.load("epinions_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([0.00082304])

In [4]:
# scaling the PageRank data
target_pagerank = target_pagerank * 10000
target_pagerank[0:10]

array([[ 8.23038636],
       [10.63970906],
       [ 3.04659061],
       [ 0.60241765],
       [ 2.1125303 ],
       [ 2.98758742],
       [ 3.50898413],
       [ 0.48461399],
       [ 1.68108362],
       [ 0.23305616]])

In [5]:
target_pagerank.shape

(75879, 1)

In [6]:
# the order 5 NFDC matrix as the feature set
matrix_1 = np.load("epinions_NDFC_matrix_r1_15_sta1_max70_rad3_inward.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [7]:
matrix_1.shape

(75879, 4, 43)

In [8]:
matrix_1 = matrix_1.reshape(matrix_1.shape[0],-1)
matrix_1.shape

(75879, 172)

# Converting the data into pytorch tenors

In [9]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([75879, 172]), torch.Size([75879, 1]))

In [10]:
# Activating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[ 8.2304],
        [10.6397],
        [ 3.0466],
        ...,
        [ 0.0287],
        [ 0.0287],
        [ 0.0287]], dtype=torch.float64, requires_grad=True)

In [11]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 55879
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([20000, 172]),
 torch.Size([55879, 172]),
 torch.Size([20000, 1]),
 torch.Size([55879, 1]))

## A function for dividing train data into batches

In [12]:
# dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # Dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [13]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

In [14]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (fc1): Linear(in_features=172, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [15]:
# Number of parameters
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print("----------------------")
print(f'Number of all parameters: \n{num_para}')

68800
400
320000
800
160000
200
12800
64
512
8
8
1
----------------------
Number of all parameters: 
563593


In [16]:
# Objective function and optimmizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the Training Loop

In [17]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    TEST LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [18]:
losses = training_loop(n_epochs=1000,
                  batch_size=400,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   0.20696609481,    TEST LOSS:   0.16867062449
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.00994841661,    TEST LOSS:   0.15092313290
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.00500511844,    TEST LOSS:   0.13885948062
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.00312423264,    TEST LOSS:   0.13202163577
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.00267366599,    TEST LOSS:   0.12856246531
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.00196691835,    TEST LOSS:   0.12445426732
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.00162177440,    TEST LOSS:   0.12367653102
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.00123870478,    TEST LOSS:   0.12227993459
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [19]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,features.shape[1])
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%100 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------------------------------------")

target 0.03434652529952794,    prediction: 0.03572341054677963
index 0:       inaccuracy: 4.009%
-----------------------------------------------------------------
target 0.22104889641503975,    prediction: 0.13247409462928772
index 100:       inaccuracy: 40.07%
-----------------------------------------------------------------
target 0.03748261911389146,    prediction: 0.053441308438777924
index 200:       inaccuracy: 42.576%
-----------------------------------------------------------------
target 0.10615686792643043,    prediction: 0.04098310321569443
index 300:       inaccuracy: 61.394%
-----------------------------------------------------------------
target 0.36478617865365537,    prediction: 0.19249698519706726
index 400:       inaccuracy: 47.23%
-----------------------------------------------------------------
target 0.11283842197458921,    prediction: 0.08317148685455322
index 500:       inaccuracy: 26.292%
-----------------------------------------------------------------
target 0

target 0.033475065225509665,    prediction: 0.032236985862255096
index 5300:       inaccuracy: 3.699%
-----------------------------------------------------------------
target 0.05730341010294005,    prediction: 0.04952671378850937
index 5400:       inaccuracy: 13.571%
-----------------------------------------------------------------
target 0.038689381972296294,    prediction: 0.051464155316352844
index 5500:       inaccuracy: 33.019%
-----------------------------------------------------------------
target 0.030871442026172403,    prediction: 0.03384663164615631
index 5600:       inaccuracy: 9.637%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 5700:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.17439431337480946,    prediction: 0.14779679477214813
index 5800:       inaccuracy: 15.251%
------------------------------------------------------------

target 0.055096218574491276,    prediction: 0.03954457491636276
index 10500:       inaccuracy: 28.226%
-----------------------------------------------------------------
target 0.05659979289518364,    prediction: 0.04282841086387634
index 10600:       inaccuracy: 24.331%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 10700:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.13195568702370636,    prediction: 0.08405956625938416
index 10800:       inaccuracy: 36.297%
-----------------------------------------------------------------
target 0.14451056230660905,    prediction: 0.17903250455856323
index 10900:       inaccuracy: 23.889%
-----------------------------------------------------------------
target 0.032675824889830986,    prediction: 0.06185746192932129
index 11000:       inaccuracy: 89.307%
-------------------------------------------------------

target 0.03644430012349156,    prediction: 0.049353353679180145
index 15600:       inaccuracy: 35.421%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 15700:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.04136519406915699,    prediction: 0.033292271196842194
index 15800:       inaccuracy: 19.516%
-----------------------------------------------------------------
target 0.19948646254710092,    prediction: 0.17264457046985626
index 15900:       inaccuracy: 13.455%
-----------------------------------------------------------------
target 1.3672471598505338,    prediction: 0.6605192422866821
index 16000:       inaccuracy: 51.69%
-----------------------------------------------------------------
target 0.12839016328764025,    prediction: 0.43573886156082153
index 16100:       inaccuracy: 239.386%
---------------------------------------------------------

target 0.046531032104645105,    prediction: 0.03404682129621506
index 20800:       inaccuracy: 26.83%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 20900:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.2577422012804544,    prediction: 0.22339434921741486
index 21000:       inaccuracy: 13.326%
-----------------------------------------------------------------
target 0.07523427434826416,    prediction: 0.06546174734830856
index 21100:       inaccuracy: 12.989%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 21200:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 21300:       inaccuracy: 3.932%
--------------------------------------------------------

...

target 0.028653960184055832,    prediction: 0.027527369558811188
index 31300:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 31400:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.05982680141256228,    prediction: 0.039080820977687836
index 31500:       inaccuracy: 34.677%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 31600:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.031547950695002595,    prediction: 0.05189631134271622
index 31700:       inaccuracy: 64.5%
-----------------------------------------------------------------
target 0.07214436107438552,    prediction: 0.06362972408533096
index 31800:       inaccuracy: 11.802%
-------------------------------------------------------

target 0.028653960184055832,    prediction: 0.027527369558811188
index 36200:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.731386547485202,    prediction: 1.0310945510864258
index 36300:       inaccuracy: 40.978%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 36400:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.03129621349408912,    prediction: 0.03103848546743393
index 36500:       inaccuracy: 0.824%
-----------------------------------------------------------------
target 0.04626452545788114,    prediction: 0.047498054802417755
index 36600:       inaccuracy: 2.666%
-----------------------------------------------------------------
target 0.029677648384407875,    prediction: 0.04461229592561722
index 36700:       inaccuracy: 50.323%
-----------------------------------------------------------

target 0.028653960184055832,    prediction: 0.027527369558811188
index 41100:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.05046970785134949,    prediction: 0.046557195484638214
index 41200:       inaccuracy: 7.752%
-----------------------------------------------------------------
target 0.06885407957138034,    prediction: 0.022699899971485138
index 41300:       inaccuracy: 67.032%
-----------------------------------------------------------------
target 0.16268181502239545,    prediction: 0.11178657412528992
index 41400:       inaccuracy: 31.285%
-----------------------------------------------------------------
target 0.14253199223186994,    prediction: 0.10667254775762558
index 41500:       inaccuracy: 25.159%
-----------------------------------------------------------------
target 0.13353249855968377,    prediction: 0.11285234242677689
index 41600:       inaccuracy: 15.487%
--------------------------------------------------------

target 0.07425437844019517,    prediction: 0.03403357416391373
index 46100:       inaccuracy: 54.166%
-----------------------------------------------------------------
target 0.05248976715063066,    prediction: 0.04903974384069443
index 46200:       inaccuracy: 6.573%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 46300:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.037381818474206435,    prediction: 0.04281053692102432
index 46400:       inaccuracy: 14.522%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 46500:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 46600:       inaccuracy: 3.932%
-------------------------------------------------------

target 0.03434652529952794,    prediction: 0.03572341054677963
index 51000:       inaccuracy: 4.009%
-----------------------------------------------------------------
target 0.04035289938151134,    prediction: 0.060974664986133575
index 51100:       inaccuracy: 51.104%
-----------------------------------------------------------------
target 0.028653960184055832,    prediction: 0.027527369558811188
index 51200:       inaccuracy: 3.932%
-----------------------------------------------------------------
target 0.03336374271276375,    prediction: 0.035712577402591705
index 51300:       inaccuracy: 7.04%
-----------------------------------------------------------------
target 0.06817806748018035,    prediction: 0.08551596850156784
index 51400:       inaccuracy: 25.43%
-----------------------------------------------------------------
target 0.21230356719812196,    prediction: 0.11560575664043427
index 51500:       inaccuracy: 45.547%
-----------------------------------------------------------

In [20]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  27.196


In [21]:
torch.save(model, "trained_FFNN_pagerank_epinions.pt")